In [ ]:
import os
import re
from lxml import etree

def get_svg_size(svg_root):
    """Get width, height from <svg> attributes or viewBox"""
    width = svg_root.get("width")
    height = svg_root.get("height")
    viewBox = svg_root.get("viewBox")

    def parse_size(value):
        if value is None:
            return None
        match = re.match(r"([\d.]+)", value)
        return float(match.group(1)) if match else None

    if viewBox:
        parts = viewBox.split()
        if len(parts) == 4:
            return float(parts[2]), float(parts[3])
    return parse_size(width), parse_size(height)

def scale_svg(svg_path, target_size=200):
    """Scale an SVG file proportionally so its largest dimension is target_size"""
    tree = etree.parse(svg_path)
    root = tree.getroot()
    nsmap = root.nsmap

    w, h = get_svg_size(root)
    if not w or not h:
        print(f"Skipping {svg_path}: can't read size")
        return

    # Only process small SVGs (< 100x100)
    if max(w, h) >= 100:
        return

    scale_factor = target_size / max(w, h)
    new_w = w * scale_factor
    new_h = h * scale_factor

    # Apply scaling transform
    for child in list(root):
        old_transform = child.get("transform", "")
        child.set("transform", f"scale({scale_factor}) {old_transform}".strip())

    # Update <svg> size attributes
    root.set("width", str(round(new_w, 2)))
    root.set("height", str(round(new_h, 2)))

    # Adjust viewBox to match new dimensions
    root.set("viewBox", f"0 0 {round(new_w, 2)} {round(new_h, 2)}")

    # Write back (overwrite)
    tree.write(svg_path, encoding="utf-8", xml_declaration=True)
    print(f"Scaled {os.path.basename(svg_path)} → {round(new_w,2)}x{round(new_h,2)}")

def process_directory(path="."):
    """Recursively process all SVGs"""
    for root_dir, _, files in os.walk(path):
        for file in files:
            if file.lower().endswith(".svg"):
                full_path = os.path.join(root_dir, file)
                scale_svg(full_path)

if __name__ == "__main__":
    process_directory(".")


TypeError: this element does not have children or attributes

In [2]:
# =========================================
# Desired output size: 500 x 500 pixels
# =========================================
import os
import re
from lxml import etree

def get_svg_size(svg_root):
    """Get width, height from <svg> attributes or viewBox"""
    width = svg_root.get("width")
    height = svg_root.get("height")
    viewBox = svg_root.get("viewBox")

    def parse_size(value):
        if value is None:
            return None
        match = re.match(r"([\d.]+)", value)
        return float(match.group(1)) if match else None

    if viewBox:
        parts = viewBox.split()
        if len(parts) == 4:
            return float(parts[2]), float(parts[3])
    return parse_size(width), parse_size(height)

def scale_svg(svg_path, target_size=500):
    """Scale an SVG file proportionally so its largest dimension is target_size"""
    try:
        tree = etree.parse(svg_path)
        root = tree.getroot()
    except Exception as e:
        print(f"⚠️  Error parsing {svg_path}: {e}")
        return

    w, h = get_svg_size(root)
    if not w or not h:
        print(f"Skipping {svg_path}: can't read size")
        return

    scale_factor = target_size / max(w, h)
    new_w = w * scale_factor
    new_h = h * scale_factor

    # Wrap everything inside a <g> element and apply scale there (cleaner & safer)
    g = etree.Element("g")
    g.set("transform", f"scale({scale_factor})")

    # Move all existing children into the <g>
    for child in list(root):
        if isinstance(child.tag, str):  # Only move real elements
            root.remove(child)
            g.append(child)
    root.append(g)

    # Update <svg> size attributes
    root.set("width", str(round(new_w, 2)))
    root.set("height", str(round(new_h, 2)))

    # Adjust viewBox to match new dimensions
    root.set("viewBox", f"0 0 {round(new_w, 2)} {round(new_h, 2)}")

    # Write back (overwrite original)
    tree.write(svg_path, encoding="utf-8", xml_declaration=True)
    print(f"✅ Scaled {os.path.basename(svg_path)} → {round(new_w,2)}x{round(new_h,2)}")

def process_directory(path=".", target_size=500):
    """Recursively process all SVGs"""
    for root_dir, _, files in os.walk(path):
        for file in files:
            if file.lower().endswith(".svg"):
                full_path = os.path.join(root_dir, file)
                scale_svg(full_path, target_size)

if __name__ == "__main__":
    process_directory(".", target_size=500)


✅ Scaled printables-banner-black.svg → 500.0x109.38
✅ Scaled printables-spool.svg → 500.0x500.0
✅ Scaled thingiverse-banner.svg → 500.0x96.77
✅ Scaled thingiverse-color-banner.svg → 500.0x60.61
